In [2]:
import torch
import scipy.signal as signal
import numpy as np
import matplotlib.pyplot as plt

In [1]:
def fft_convolve_images(image, kernel_stack, summation = False):
    # image is the image to be convolved
    # kernel_stack is the stack of kernels to be convolved with the image
    
    # The image and the kernel stack should be the same size
    # The kernel stack should be in the format [depth, x, y]
    return_tensor = torch.fft.ifftshift(torch.fft.ifft2(torch.mul(torch.fft.fft2(image), torch.fft.fft2(torch.div(kernel_stack, torch.sum(kernel_stack, dim = (1,2)).unsqueeze(1).unsqueeze(2)))), dim = (1,2))) # Convolve the image with the kernel stack
        
    if summation == True:
        return_tensor = torch.sum(return_tensor, 0) # Sum all of the convolutions
    
    return return_tensor

In [ ]:
torch_convolution_1tovol = torch.nn.Conv2d(1, len(PSF[]), kernel_size = (len(PSF[0,:,0]), len(PSF[0,0,:])), stride = 1, padding = 0, bias = False).to(device)
torch_convolution_volto1 = torch.nn.Conv2d(len(PSF[:,0,0]), 1, kernel_size = (len(PSF[0,:,0]), len(PSF[0,0,:])), stride = 1, padding = 0, bias = False).to(device)

In [ ]:
psf = torch.load